In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dmyplant2
dmyplant2.cred()
mp = dmyplant2.MyPlant(3600)

In [2]:
dval = dmyplant2.Validation.load_def_csv("input2.csv")
dval;

In [3]:
i = 2
vale = dval.iloc[i][['serialNumber','Validation Engine','val start']]

In [4]:
e=dmyplant2.Engine.from_sn(mp,vale['serialNumber'], valstart=vale['val start'])
pd.DataFrame.from_dict(e.dash, orient='index').T

,Name,Engine ID,Design Number,Engine Type,Engine Version,P,P_nom,BMEP,serialNumber,id,Count_OpHour,val start,oph@start,oph parts,LOC
0,ROCHE PENZBERG KWKK,M01,U233,624,H02,24,4489.3,24.0,1184199,108532,39529.0,2020-04-27,0,39529,0.0185


In [7]:
from dfsm import msgFSM
fsm = msgFSM(e, p_to="2020-02-01")
fsm.run() # run the Finite State Machine
fsm.store()

4460448475it [00:08, 508340461.52it/s]   


In [8]:
%load_ext autoreload
%autoreload 2
from dfsm import msgFSM
fsm = msgFSM(e, frompickle=True)
fsm._dot('FSM.dot')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
fsm._runlog;

In [ ]:
rdf = pd.DataFrame(fsm._starts)
rda = rdf
#rda = rdf[((rdf['mode'] == 'AUTO') & (rdf['success'] == True))]
#rda = rdf[((rdf['mode'] == 'AUTO') & (rdf['success'] == True) & (rdf['synchronize'] < 300.0) & (rdf['start-preparation'] < 300.0))]
#rda = rdf[((rdf['success'] == True) & (rdf['start-preparation'] < 1000.0))]
rda = rdf[((rdf['success'] == True))]
rda.round(2)

In [ ]:
rda.describe().round(2).T

In [ ]:
rda[['start-preparation','starter','hochlauf','idle','synchronize','load-ramp']].hist(figsize = (20,12), bins=100);
#rda.hist(figsize = (20,12), bins=100);

In [ ]:
rda.plot.box(subplots=True, grid = True, figsize=(24,15), layout=(2,4));

In [ ]:
rda.plot.box(subplots=True, grid = True, figsize=(24,15), layout=(2,4), sym='');

In [ ]:
round((rdf[rdf.success].shape[0] / rdf.shape[0]) * 100,1)

In [ ]:
phases = [['start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation'],'start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation']
fig = plt.figure();
color = 'red'

for phase in phases:
    al = fsm.alarms_pareto(phase)[:20]
    if not al.empty:
        al['msg'] = al['msg'] + ' (' + al['name'] + ')'
        al.set_index('msg').sort_values(by = "anz",ascending=True).plot.barh(y=['anz'],figsize=(16,len(al) / 3.8), color=[color], position = 1.0, grid=True, title=' | '.join(phase) if type(phase) == list else phase);
        plt.plot();


In [ ]:
phases = [['start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation'],'start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation']
fig = plt.figure();
color = 'orange'

for phase in phases:
    al = fsm.warnings_pareto(phase)[:20]
    if not al.empty:
        al['msg'] = al['msg'] + ' (' + al['name'] + ')'
        al.set_index('msg').sort_values(by = "anz",ascending=True).plot.barh(y=['anz'],figsize=(16,len(al) / 3.8), color=[color], position = 1.0, grid=True, title=' | '.join(phase) if type(phase) == list else phase);
        plt.plot();


In [ ]:
#fsm.save_messages('msg3.txt')

In [ ]:
#fsm.store()